<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

# Описание данных

- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — сколько лет человек является клиентом банка
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата
- Целевой признак
- Exited — факт ухода клиента

## Подготовка данных

Импортируем библиотеки и посмторим на данные

In [90]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, roc_curve, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier


import warnings
warnings.filterwarnings("ignore")

In [24]:
try:
    data = pd.read_csv('C:/Users/School252/Downloads/dataset.csv')
except:
    data = pd.read_csv('/datasets/Churn.csv')

Общая информация

In [25]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Приведем названия столбцов в подобающий вид

In [26]:
data.columns = ['row_number', 'customer_id', 'surname', 'credit_score', 'geography', 
                'gender', 'age', 'tenure', 'balance', 'num_of_product', 'has_cr_card', 
                'is_active_member', 'estimated_salary', 'exited']
data.head()

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_product,has_cr_card,is_active_member,estimated_salary,exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Уберем столбцы, которые не несут важной информации

In [27]:
data = data.drop(['row_number', 'surname', 'customer_id'], axis=1)
data.head()

,credit_score,geography,gender,age,tenure,balance,num_of_product,has_cr_card,is_active_member,estimated_salary,exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Описание данных

In [28]:
data.describe()

,credit_score,age,tenure,balance,num_of_product,has_cr_card,is_active_member,estimated_salary,exited
count,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Из таблицы можно увидеть большой разброс между минимальным и максимальным показателем

In [29]:
data.dtypes

credit_score          int64
geography            object
gender               object
age                   int64
tenure              float64
balance             float64
num_of_product        int64
has_cr_card           int64
is_active_member      int64
estimated_salary    float64
exited                int64
dtype: object

В данной таблице присутсвует 2 категориальных признака. Воспользуемся прямым кодированием и избежим дамми-ловушек

In [30]:
data = pd.get_dummies(data, drop_first=True)
data.head()

,credit_score,age,tenure,balance,num_of_product,has_cr_card,is_active_member,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


Вывод:
- изменил названия столбцов
- увидел общую информацию
- убрал ненужные столбцы
- воспользовалься get_dummies

In [31]:
data.isna().sum()

credit_score           0
age                    0
tenure               909
balance                0
num_of_product         0
has_cr_card            0
is_active_member       0
estimated_salary       0
exited                 0
geography_Germany      0
geography_Spain        0
gender_Male            0
dtype: int64

In [36]:
# Заменю на медианные значения
data['tenure'] = data['tenure'].fillna(data['tenure'].median())
data.isna().sum()

credit_score         0
age                  0
tenure               0
balance              0
num_of_product       0
has_cr_card          0
is_active_member     0
estimated_salary     0
exited               0
geography_Germany    0
geography_Spain      0
gender_Male          0
dtype: int64

In [39]:
data.duplicated().sum()

0

## Исследование задачи

Целевой и обычные признаки

In [40]:
# Разделим на признаки
features = data.drop('exited', axis=1)
target = data['exited']

In [82]:
features_train, features_validtest, target_train, target_validtest = train_test_split(features,
                                                    target,
                                                    train_size=0.6,
                                                    random_state=12345, 
                                                    stratify=data['exited'])

In [84]:
features_valid, features_test, target_valid, target_test = train_test_split(features_validtest,
                                                    target_validtest,
                                                    train_size=0.5,
                                                    random_state=12345)

In [43]:
print(f'Тестовая выборка составляет: {features_test.shape[0]/data.shape[0]*100} % от исходного датасета df')
print(f'Валидационная выборка составляет: {features_valid.shape[0]/data.shape[0]*100} % от исходного датасета df')
print(f'Обучающая выборка составляет: {features_train.shape[0]/data.shape[0]*100} % от исходного датасета')

Тестовая выборка составляет: 20.0 % от исходного датасета df
Валидационная выборка составляет: 20.0 % от исходного датасета df
Обучающая выборка составляет: 60.0 % от исходного датасета


Датасет поделил 3:1:1

In [44]:
# Значения которы нужно масштабировать
numeric = ['credit_score', 'age', 'tenure', 'balance', 'num_of_product', 'estimated_salary']

In [45]:
# Масштабирование данных
scaler = StandardScaler()
scaler.fit(features_train[numeric])

StandardScaler()

In [46]:
# Масштабируем признаки
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])
features_train.head()

,credit_score,age,tenure,balance,num_of_product,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
7479,-0.886751,-0.373192,1.082277,1.232271,-0.891560,1,0,-0.187705,0,1,1
3411,0.608663,-0.183385,1.082277,0.600563,-0.891560,0,0,-0.333945,0,0,0
6027,2.052152,0.480939,-0.737696,1.027098,0.830152,0,1,1.503095,1,0,1
1247,-1.457915,-1.417129,0.354288,-1.233163,0.830152,1,0,-1.071061,0,0,1
3716,0.130961,-1.132419,-1.101690,1.140475,-0.891560,0,0,1.524268,1,0,0


In [50]:
# Функция для accaracy 3 моделей
def all_models_accuracy(features_train, target_train, features_valid, target_valid):
    model_DTC = DecisionTreeClassifier(random_state=12345)
    DTC_score = model_DTC.fit(features_train, target_train).score(features_valid, target_valid)
    
    model_RFC = RandomForestClassifier(max_depth=11, n_estimators=42, random_state=12345)
    RFC_score = model_RFC.fit(features_train, target_train).score(features_valid, target_valid)
    
    model_LgR = LogisticRegression(solver = 'liblinear', random_state=12345)
    LgR_score = model_LgR.fit(features_train, target_train).score(features_valid, target_valid)
    print("Точность: " "дерево решений", DTC_score, "случайный лес ", RFC_score, "логистческая регрессия", LgR_score)

In [51]:
all_models_accuracy(features_train, target_train, features_valid, target_valid)

Точность: дерево решений 0.7935 случайный лес  0.863 логистческая регрессия 0.802


In [52]:
target_train.value_counts(normalize = 1)

0    0.800667
1    0.199333
Name: exited, dtype: float64

В выборке 0--80%, 1--20%

In [53]:
# Функция показывает соотношение ответов моделей
def all_models_share(features_train, target_train, features_valid, target_valid):
    
    model_DTC = DecisionTreeClassifier(random_state=12345)
    model_DTC.fit(features_train, target_train)
    DTC_share = pd.Series(model_DTC.predict(features_valid)).value_counts(normalize = 1)
    
    model_RFC = RandomForestClassifier(random_state=12345, n_estimators = 100)
    model_RFC.fit(features_train, target_train)
    RFC_share = pd.Series(model_RFC.predict(features_valid)).value_counts(normalize = 1)
    
    model_LgR = LogisticRegression(solver = 'liblinear', random_state=12345)
    model_LgR.fit(features_train, target_train)
    LgR_share = pd.Series(model_LgR.predict(features_valid)).value_counts(normalize = 1)
    
    print("Дерево решений\n", DTC_share, "Случайный лес\n", 
          RFC_share, "Логистческая регрессия\n", LgR_share , end='')

In [54]:
# Функция для полноты, точности и f1
def rec_prec_f1(target_valid, prediction):
    print("Полнота" , recall_score(target_valid, prediction))
    print("Точность", precision_score(target_valid, prediction))
    print("F1-мера", f1_score(target_valid, prediction))

In [55]:
all_models_share(features_train, target_train, features_valid, target_valid)

Дерево решений
 0    0.7955
1    0.2045
dtype: float64 Случайный лес
 0    0.8745
1    0.1255
dtype: float64 Логистческая регрессия
 0    0.913
1    0.087
dtype: float64

Самая высокая точность у логистичесой модели, затем случаный лес и дерево решений

Так как выборки с дисбалансом большинство ответов 0

# Случайный лес

In [92]:
model_RFC = RandomForestClassifier(random_state=12345, n_estimators = 100)
model_RFC.fit(features_train, target_train)
RFC_prediction = model_RFC.predict(features_valid)
print(confusion_matrix(target_valid, RFC_prediction))

[[1542   67]
 [ 208  183]]


Много значений TN, но классы не сбалансированны

In [93]:
rec_prec_f1(target_valid, RFC_prediction)

Полнота 0.4680306905370844
Точность 0.732
F1-мера 0.5709828393135725


In [94]:
RFC_probabilities_one_valid = model_RFC.predict_proba(features_valid)[:, 1]
auc_roc_RFC = roc_auc_score(target_valid, RFC_probabilities_one_valid)
auc_roc_RFC

0.8472427315023072

Точность и качество за счет TP

# Дерево решений

In [95]:
model_DTC = DecisionTreeClassifier(random_state=12345)
model_DTC.fit(features_train, target_train)
DTC_prediction = model_DTC.predict(features_valid)
print(confusion_matrix(target_valid, DTC_prediction))

[[1383  226]
 [ 206  185]]


In [96]:
rec_prec_f1(target_valid, DTC_prediction)

Полнота 0.4731457800511509
Точность 0.45012165450121655
F1-мера 0.4613466334164588


In [97]:
DTC_probabilities_one_valid = model_DTC.predict_proba(features_valid)[:, 1]
auc_roc_DTC = roc_auc_score(target_valid, DTC_probabilities_one_valid)
auc_roc_DTC

0.6663429335308583

Высокая доля FP, низкое значение f1 и точности

# Логистическая регрессия

In [98]:
model_LgR = LogisticRegression(solver = 'liblinear', random_state=12345)
model_LgR.fit(features_train, target_train)
LgR_prediction = model_LgR.predict(features_valid)
print(confusion_matrix(target_valid, LgR_prediction))

[[1574   35]
 [ 376   15]]


In [99]:
rec_prec_f1(target_valid, LgR_prediction)

Полнота 0.03836317135549872
Точность 0.3
F1-мера 0.06802721088435375


In [100]:
LgR_probabilities_one_valid = model_LgR.predict_proba(features_valid)[:, 1]
auc_roc_LgR = roc_auc_score(target_valid, LgR_probabilities_one_valid)
auc_roc_LgR

0.6760199580683464

Очень низкие полнота и точность, качество тоже не высокое

## Борьба с дисбалансом

In [101]:
# Функция для баланса выборки
def upsample(features, target, repeat, upsampled_сlass):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    if upsampled_сlass == 0:
        features_upsampled = pd.concat([features_zeros]* repeat + [features_ones] )
        target_upsampled = pd.concat([target_zeros]* repeat + [target_ones] )
        features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
        
    elif upsampled_сlass == 1:
        features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
        target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
        features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    else:
        features_upsampled = 0
        target_upsampled = 0
        
    return features_upsampled, target_upsampled

In [102]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 4, 1)

In [103]:
print(target_upsampled.value_counts(normalize = 1))
print(target_upsampled.shape)

1    0.50569
0    0.49431
Name: exited, dtype: float64
(9666,)


Теперь значения примерно пополам

In [104]:
all_models_accuracy(features_train, target_train, features_valid, target_valid)

Точность: дерево решений 0.784 случайный лес  0.867 логистческая регрессия 0.7945


In [105]:
all_models_accuracy(features_upsampled, target_upsampled, features_valid, target_valid)

Точность: дерево решений 0.7995 случайный лес  0.832 логистческая регрессия 0.6735


Все показатели немного подросли, обучим модеи на этих данных

# Случайный лес

In [106]:
model_RFC_upsampled = RandomForestClassifier(random_state=12345, n_estimators = 100)
model_RFC_upsampled.fit(features_upsampled, target_upsampled)
RFC_prediction_upsampled = model_RFC.predict(features_valid)
rec_prec_f1(target_valid, RFC_prediction_upsampled)

Полнота 0.4680306905370844
Точность 0.732
F1-мера 0.5709828393135725


In [107]:
RFC_upsampled_valid = model_RFC_upsampled.predict_proba(features_valid)[:, 1]
auc_roc_RFC = roc_auc_score(target_valid, RFC_upsampled_valid)
auc_roc_RFC

0.8448918249170665

Точность и качество выросли

# Дерево решений

In [108]:
model_DTC_upsampled = DecisionTreeClassifier(random_state=12345, max_depth=11)
model_DTC_upsampled.fit(features_upsampled, target_upsampled)
DTC_prediction_upsampled = model_DTC.predict(features_valid)
rec_prec_f1(target_valid, DTC_prediction_upsampled)

Полнота 0.4731457800511509
Точность 0.45012165450121655
F1-мера 0.4613466334164588


In [109]:
DTC_upsampled_valid = model_DTC_upsampled.predict_proba(features_valid)[:, 1]
auc_roc_DTC = roc_auc_score(target_valid, DTC_upsampled_valid)
auc_roc_DTC

0.7208238822861811

Так же наблюдается прирост

# Логистическая регрессия

In [110]:
model_LgR_upsampled = LogisticRegression(solver = 'liblinear', random_state=12345)
model_LgR_upsampled.fit(features_upsampled, target_upsampled)
LgR_prediction_upsampled = model_LgR.predict(features_valid)
rec_prec_f1(target_valid, LgR_prediction_upsampled)

Полнота 0.03836317135549872
Точность 0.3
F1-мера 0.06802721088435375


In [111]:
LgR_upsampled_valid = model_LgR_upsampled.predict_proba(features_valid)[:, 1]
auc_roc_LgR = roc_auc_score(target_valid, LgR_upsampled_valid)
auc_roc_LgR

0.7334796755462797

Показатели возрасли

Вывод: Наблюдается прирост в кадой модели, лучше всего себя показывает случайный лес с f1 = 0.57
- баланс классов
- выбор наилучшей модели

## Тестирование модели

In [112]:
model_RFC_final = RandomForestClassifier(
    bootstrap = True, class_weight = 'balanced', max_depth= 10,  n_estimators = 60, random_state=12345)
model_RFC_final.fit(features_upsampled, target_upsampled)

RandomForestClassifier(class_weight='balanced', max_depth=10, n_estimators=60,
                       random_state=12345)

In [113]:
model_RFC_final_prediction = model_RFC_final.predict(features_test)
rec_prec_f1(target_test, model_RFC_final_prediction)

Полнота 0.6863207547169812
Точность 0.5938775510204082
F1-мера 0.6367614879649891


auc_roc для модели на тестовой выборке

In [114]:
model_RFC_final_valid = model_RFC_final.predict_proba(features_test)[:, 1]
auc_roc_RFC = roc_auc_score(target_test, model_RFC_final_valid)
auc_roc_RFC

0.8703219279762476

Создадим константную модель

In [115]:
target_predict_constant = pd.Series([0]*len(target_test))
target_predict_constant.value_counts()

0    2000
dtype: int64

In [116]:
#Сравним показатель точности (accuracy_score) константной модели и финальной
print('accuracy_score константой модели:', accuracy_score(target_valid, target_predict_constant))
print('accuracy_score финальной модели:', accuracy_score(target_test, model_RFC_final_prediction))
#Дополнительно сравним AUC-ROC — единственный параметр подающийся сравнению, потому что константная подель содержит только негативные ответы
print('AUC-ROC константой модели:', roc_auc_score(target_valid, target_predict_constant))
print('AUC-ROC финальной модели:', roc_auc_score(target_test, model_RFC_final_valid))

accuracy_score константой модели: 0.8045
accuracy_score финальной модели: 0.834
AUC-ROC константой модели: 0.5
AUC-ROC финальной модели: 0.8703219279762476


Финальные показатели у одели лучше чем у константной

Вывод: Построил модель с предельно большим значением F1-меры. Сравнивал данный показатель с AUC_ROC. Загрузил и подготовил данные. Исследовал баланс классов, обучил модель без учёта дисбаланса. Улучшил качество модели, учитывая дисбаланс классов. Обучил разные модели и нашел лучшую. Провел финальное тестирование. Итоговое значеие f1 = 0.61